# Data Investigation

In [2]:
import pandas as pd
import cbpro
import sqlite3
import matplotlib.pyplot as plt

In [10]:
# Create database connection
# Create connection
conn = sqlite3.connect('../db/crypto-analysis.db')
c = conn.cursor()

In [11]:
# Date of all time high for each coin
pd.read_sql(
    '''
    SELECT date, symbol, max(high)
    FROM coinbase
    GROUP BY symbol
    ORDER BY 3 DESC
    ''', conn
)

,date,symbol,max(high)
0,2021-05-12,YFI,96000.00000
1,2021-04-14,BTC,64899.00000
2,2021-04-14,WBTC,64871.98000
3,2017-12-20,BCH,9500.00000
4,2021-05-03,MKR,6391.78050
...,...,...,...
83,2021-08-23,CHZ,0.41600
84,2021-08-06,ACH,0.24100
85,2021-03-28,ANKR,0.21764
86,2021-08-12,IOTX,0.13860


In [13]:
# Date of all time low for each coin
pd.read_sql(
    '''
    SELECT date, symbol, min(low)
    FROM coinbase
    GROUP BY symbol
    ORDER BY 3 DESC
    ''', conn
)

,date,symbol,min(low)
0,2020-10-20,WBTC,11890.240000
1,2020-11-05,YFI,7452.490000
2,2021-08-12,QUICK,576.000000
3,2020-06-27,MKR,430.181200
4,2021-07-29,FARM,101.050000
...,...,...,...
83,2017-04-15,BTC,0.060000
84,2021-06-22,ANKR,0.047590
85,2021-07-20,AMP,0.038000
86,2020-03-13,XLM,0.026139


Shocking to see that BTC hit $0.06 April 15, 2017. Could that really be true? Let's take a closer look at the historical prices around that date...

In [16]:
pd.read_sql(
    '''
    SELECT *
    FROM coinbase
    WHERE symbol = 'BTC' AND 
        (date BETWEEN '2017-04-10' AND '2017-04-20')
    ''', conn 
)

,symbol,date,high,low,open,close,volume
0,BTC,2017-04-20,1240.60,1212.01,1214.22,1236.15,7747.225097
1,BTC,2017-04-19,1215.32,1191.09,1201.88,1214.21,6273.615376
2,BTC,2017-04-18,1206.65,1189.76,1190.95,1201.94,6269.909265
3,BTC,2017-04-17,1197.60,1170.85,1177.63,1189.91,6548.066563
4,BTC,2017-04-16,1189.93,1171.70,1178.84,1177.99,3084.525039
5,BTC,2017-04-15,1190.99,0.06,1173.13,1178.85,3972.003146
6,BTC,2017-04-14,1196.92,1170.14,1177.05,1173.74,5666.314944
7,BTC,2017-04-13,1219.49,1148.98,1214.68,1177.05,9426.537457
8,BTC,2017-04-12,1227.86,1209.61,1224.00,1214.17,6494.151562
9,BTC,2017-04-11,1234.00,1200.00,1210.00,1223.99,5481.083298


Looks like a data error. We can compare this value to what we picked up from the lunarCRUSH API...

We can also check to see if there are similar data errors for any other coins...

In [24]:
pd.read_sql(
    '''
    SELECT date, symbol, high, low, (high-low)/low AS pct_change
    FROM coinbase
    WHERE pct_change > 2
    ORDER BY pct_change DESC
    ''', conn
)

,date,symbol,high,low,pct_change
0,2017-04-15,BTC,1190.9900,0.060000,19848.833333
1,2017-06-21,ETH,352.0000,0.100000,3519.000000
2,2021-07-15,CLV,2.8000,0.510000,4.490196
3,2021-05-09,RLC,19.8500,4.934000,3.023105
4,2021-08-06,ACH,0.2410,0.060101,3.009917
5,2021-04-21,FORTH,92.0900,25.000000,2.683600
6,2021-07-16,CLV,1.4500,0.420000,2.452381
7,2021-08-05,ACH,0.0889,0.026871,2.308399
8,2020-05-22,OMG,4.5542,1.485000,2.066801
9,2017-12-20,BCH,9500.0000,3100.000000,2.064516


As expected, BTC on 4/15/17 shows up but we also see ETH on 6/21/17. Need to check against lunarCRUSH data.